In [1]:
import pandas as pd
import sys
from datetime import datetime
sys.path.append('..')

In [2]:
data_in_sample = pd.read_excel("../data/Task_data.xlsx", sheet_name="in-sample", engine='openpyxl')
data_out_of_sample = pd.read_excel("../data/Task_data.xlsx", sheet_name="out-of-sample", engine='openpyxl')
data_exog = pd.read_excel("../data/Task_data.xlsx", sheet_name="macro", engine='openpyxl')

In [3]:
data_in_sample.shape, data_out_of_sample.shape, data_exog.shape 

((165086, 23), (17532, 22), (62, 22))

In [4]:
data_in_sample["date"] = data_in_sample["date"].astype("datetime64[ns]")
data_out_of_sample["date"] = data_out_of_sample["date"].astype("datetime64[ns]")
data_exog["date_str"] = pd.to_datetime(data_exog["date_str"], format='%Y%m%d').astype("datetime64[ns]")

In [5]:
data_in_sample.rename({'outstanding_volume': "volume_schedule"}, axis=1, inplace=True)

In [6]:
data_in_sample["OPEN_DATE"] = data_in_sample["OPEN_DATE"].apply(lambda x: str(x)[:4])

In [7]:
data_out_of_sample["OPEN_DATE"] = data_out_of_sample["OPEN_DATE"].apply(lambda x: str(x)[:4])

In [8]:
data_in_sample.columns ^ data_out_of_sample.columns

<ipython-input-8-2150bf5380a5>:1: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead
  data_in_sample.columns ^ data_out_of_sample.columns


Index(['prepaid_amount'], dtype='object')

In [9]:
data_exog = data_exog[[col for col in data_exog.columns if not col.startswith("Unnamed")]]
data_exog['avg_monthly_product_client_rate_mtg_grn'] = data_exog['avg_monthly_product_client_rate_mtg_grn'].interpolate(method='backfill', limit_direction="backward")

In [10]:
fill_strategy = {"datetime64[ns]", "int64", "float64", "object"}
for column in data_in_sample.columns:
    if column in ['prepaid_amount', 'volume_schedule']:
        continue
    elif data_in_sample[column].dtype == "datetime64[ns]":
        continue
    elif data_in_sample[column].dtype == "int64": 
        fill_value = data_in_sample[column].median()
        data_in_sample[column] = data_in_sample[column].fillna(fill_value)
        data_out_of_sample[column] = data_out_of_sample[column].fillna(fill_value)
    elif data_in_sample[column].dtype == "float64": 
        fill_value = data_in_sample[column].median()
        data_in_sample[column] = data_in_sample[column].fillna(fill_value)
        data_out_of_sample[column] = data_out_of_sample[column].fillna(fill_value)
    elif data_in_sample[column].dtype == "object": 
        fill_value = data_in_sample[column].mode()[0]
        data_in_sample[column] = data_in_sample[column].fillna(fill_value)
        data_out_of_sample[column] = data_out_of_sample[column].fillna(fill_value)    

In [12]:
data_in_sample.isnull().sum()

date                        0
date_str                    0
id                          0
years_to_matur              0
age_owner_years             0
original_matur_years        0
client_rate                 0
original_volume             0
age_loan_years              0
volume_schedule             0
planned_installments        0
prepaid_amount              0
type                        0
RES_ADDR_CITY               0
EDUCATION                   0
NUMBER_OF_FAMILY_MEMBERS    0
RESIDENTAL_STATUS           0
MARITAL_STATUS              0
FIXED_MONTHLY_EXPENSES      0
Flat_House                  0
OPEN_DATE                   0
INCOME_houshold             0
dpd                         0
dtype: int64

In [13]:
data_in_sample = pd.merge(data_in_sample, data_exog, left_on="date", right_on="date_str")
data_out_of_sample = pd.merge(data_out_of_sample, data_exog, left_on="date", right_on="date_str")

In [14]:
del data_in_sample['date_str_x']
del data_in_sample['date_str_y']
del data_out_of_sample['date_str_x']
del data_out_of_sample['date_str_y']

In [15]:
data_in_sample

,date,id,years_to_matur,age_owner_years,original_matur_years,client_rate,original_volume,age_loan_years,volume_schedule,planned_installments,...,avg_monthly_salary_enterprise_index,wheat_purchase_price_index,milk_purchase_price_index,production_price_energy_index,production_price_water_supply_index,inflation,inflation_apartment_usage,new_flats,economy_index,economy_index_real_estate
0,2016-10-31,1,11.49,44.22,25.01,0.0342,51850.00,13.53,33397.66,196.04,...,103.7,92.3,107.4,98.2,102.3,99.8,99.6,16620,-9.1,12.5
1,2016-10-31,2,6.52,56.79,21.93,0.0342,155550.00,15.41,86282.04,623.61,...,103.7,92.3,107.4,98.2,102.3,99.8,99.6,16620,-9.1,12.5
2,2016-10-31,3,9.37,43.99,25.02,0.0432,70516.00,15.64,39803.26,282.97,...,103.7,92.3,107.4,98.2,102.3,99.8,99.6,16620,-9.1,12.5
3,2016-10-31,4,6.42,40.20,20.01,0.0850,31110.00,13.59,15952.79,155.51,...,103.7,92.3,107.4,98.2,102.3,99.8,99.6,16620,-9.1,12.5
4,2016-10-31,5,5.44,55.12,19.27,0.0279,336431.43,13.83,35330.31,494.01,...,103.7,92.3,107.4,98.2,102.3,99.8,99.6,16620,-9.1,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165081,2016-09-30,4655,4.25,49.42,5.00,0.0730,31110.00,0.75,9569.41,159.49,...,104.4,93.3,102.8,97.9,101.7,99.5,99.5,12665,-5.4,10.8
165082,2016-09-30,4656,2.83,48.84,3.00,0.0530,22693.45,0.17,21525.52,586.27,...,104.4,93.3,102.8,97.9,101.7,99.5,99.5,12665,-5.4,10.8
165083,2016-09-30,4657,4.31,49.34,4.98,0.0699,15555.00,0.67,13773.38,232.53,...,104.4,93.3,102.8,97.9,101.7,99.5,99.5,12665,-5.4,10.8
165084,2016-09-30,4658,1.00,50.67,3.01,0.0730,9125.60,2.00,3266.04,524.72,...,104.4,93.3,102.8,97.9,101.7,99.5,99.5,12665,-5.4,10.8


In [16]:
data_out_of_sample

,date,id,years_to_matur,age_owner_years,original_matur_years,client_rate,original_volume,age_loan_years,volume_schedule,planned_installments,...,avg_monthly_salary_enterprise_index,wheat_purchase_price_index,milk_purchase_price_index,production_price_energy_index,production_price_water_supply_index,inflation,inflation_apartment_usage,new_flats,economy_index,economy_index_real_estate
0,2020-03-31,1,8.07,47.63,25.01,0.0342,51850.00,16.94,24802.77,223.30,...,101.8,89.6,99.3,103.3,105.8,104.6,107.5,15580,-1.9,6.4
1,2020-03-31,9,2.42,68.03,18.48,0.0260,125341.39,16.06,5617.59,184.33,...,101.8,89.6,99.3,103.3,105.8,104.6,107.5,15580,-1.9,6.4
2,2020-03-31,10,6.41,49.62,20.06,0.0259,285175.00,13.66,85393.60,1109.02,...,101.8,89.6,99.3,103.3,105.8,104.6,107.5,15580,-1.9,6.4
3,2020-03-31,18,1.52,69.47,15.19,0.0269,312033.30,13.67,38523.61,1988.04,...,101.8,89.6,99.3,103.3,105.8,104.6,107.5,15580,-1.9,6.4
4,2020-03-31,19,16.83,43.08,30.42,0.0258,177223.30,13.59,126221.72,494.27,...,101.8,89.6,99.3,103.3,105.8,104.6,107.5,15580,-1.9,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527,2021-02-28,5351,18.44,39.97,20.36,0.0362,259250.00,1.92,241309.99,724.69,...,102.1,124.4,108.6,101.2,103.1,102.4,106.1,16723,-17.7,-4.2
17528,2021-02-28,5352,18.22,31.79,20.13,0.0356,207400.00,1.92,194039.20,676.54,...,102.1,124.4,108.6,101.2,103.1,102.4,106.1,16723,-17.7,-4.2
17529,2021-02-28,5353,8.52,36.67,10.44,0.0373,103700.00,1.92,87685.95,519.88,...,102.1,124.4,108.6,101.2,103.1,102.4,106.1,16723,-17.7,-4.2
17530,2021-02-28,5354,28.23,30.97,30.14,0.0356,362950.00,1.92,350829.79,684.04,...,102.1,124.4,108.6,101.2,103.1,102.4,106.1,16723,-17.7,-4.2


In [17]:
train_cutoff = datetime.strptime("2018-01-31", "%Y-%m-%d")
val_cutoff = datetime.strptime("2019-02-28", "%Y-%m-%d")

In [18]:
data_train = data_in_sample.query(f"date<='{train_cutoff}'")
data_val = data_in_sample.query(f"date>'{train_cutoff}' & date<='{val_cutoff}'")
data_test = data_in_sample.query(f"date>'{val_cutoff}'")

In [19]:
data_train['date'].min(), data_train['date'].max(), data_train.shape

(Timestamp('2016-01-31 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 (101960, 39))

In [20]:
data_val['date'].min(), data_val['date'].max(), data_val.shape

(Timestamp('2018-02-28 00:00:00'),
 Timestamp('2019-02-28 00:00:00'),
 (38765, 39))

In [21]:
data_test['date'].min(), data_test['date'].max(), data_test.shape

(Timestamp('2019-03-31 00:00:00'),
 Timestamp('2020-02-29 00:00:00'),
 (24361, 39))

In [22]:
data_train.to_csv("../data/data_train.csv")
data_val.to_csv("../data/data_val.csv")
data_test.to_csv("../data/data_test.csv")
data_out_of_sample.to_csv("../data/data_out_of_sample.csv")